### Load libraries

In [14]:
from pyspark.sql.functions import col
from pyspark.sql.functions import pandas_udf
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
import datetime as dt
from pyspark.sql.types import StructType, StructField, StringType, DoubleType
import joblib
import os
from xgboost import XGBRegressor
import xgboost as xgb

StatementMeta(, 6b210051-0b76-4989-9dc5-1bf0aaec88bd, 16, Finished, Available, Finished)

### Load Trained Model


In [15]:
# Load model and MAE
import joblib

model_dir = "/lakehouse/default/Files/models/user_estimates"
os.makedirs(model_dir, exist_ok=True)
model_path = os.path.join(model_dir, "user_estimates_model.joblib")
artifact = joblib.load(model_path)

enhanced_model = artifact['model']
mae_test = artifact['mae_test']

StatementMeta(, 6b210051-0b76-4989-9dc5-1bf0aaec88bd, 17, Finished, Available, Finished)

## Predict for All Rows in imputed_fact_live_times

In [18]:
simple_features = [
    'PopularTimesLivePercent', 'ParkSize', 'Score', 'ClassificationScore', 'BusStopHa', 'PopHa'
]

enhanced_features = simple_features + [
    'Temp', 'IsRaining', 'IsSnowing'
]

StatementMeta(, 6b210051-0b76-4989-9dc5-1bf0aaec88bd, 20, Finished, Available, Finished)

In [19]:
# Step 1: Read tables
fact_df = spark.read.table("imputed_fact_live_times").filter("ParkKey != -1") 
dim_park = spark.read.table("dim_park_attributes")
dim_date = spark.read.table("dim_date")

# Step 2: Join fact with dim_park_attributes (by ParkKey)
fact_plus_park = fact_df.join(
    dim_park.select("ParkKey", "ParkSize", "Score", "BusStopHa", "PopHa", "Classification"),
    on="ParkKey",
    how="left"
)

# Step 3: Join with dim_date (by DateKey)
fact_plus_all = fact_plus_park.join(
    dim_date.select("DateKey", "Month"),
    on="DateKey",
    how="left"
)

# Step 4: Convert to pandas
df_all = fact_plus_all.toPandas()

# Step 5: Map Classification to ClassificationScore
classification_score_map = {
    'Destination': 7,
    'Community': 5,
    'Neighbourhood': 3,
    'Local': 1,
    'Urban Plaza': 1
}
df_all['ClassificationScore'] = df_all['Classification'].map(classification_score_map)

# Step 6: Build final features
df_features = df_all[['FactLiveKey', 'ParkKey', 'DateKey', 'HourKey'] + enhanced_features].copy()

StatementMeta(, 6b210051-0b76-4989-9dc5-1bf0aaec88bd, 21, Finished, Available, Finished)

In [20]:
# Load data
display(df_features.head())

StatementMeta(, 6b210051-0b76-4989-9dc5-1bf0aaec88bd, 22, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, a31c34c9-4f8f-4590-b1c0-0984e941e682)

In [21]:
df_features.shape

StatementMeta(, 6b210051-0b76-4989-9dc5-1bf0aaec88bd, 23, Finished, Available, Finished)

(359762, 13)

In [22]:
enhanced_features

StatementMeta(, 6b210051-0b76-4989-9dc5-1bf0aaec88bd, 24, Finished, Available, Finished)

['PopularTimesLivePercent',
 'ParkSize',
 'Score',
 'ClassificationScore',
 'BusStopHa',
 'PopHa',
 'Temp',
 'IsRaining',
 'IsSnowing']

In [23]:
from tqdm import tqdm

# # Use MAE from test set as uncertainty buffer
# mae_test = mean_absolute_error(y_enhanced_test, grid_enhanced.best_estimator_.predict(X_enhanced_test))

def predict_visitors(model_pipeline, input_df):
    """
    Predict visitor count from a trained pipeline using XGBoost.
    Expects a DataFrame with 1 row.
    """
    X_transformed = model_pipeline.named_steps['prep'].transform(input_df)
    pred = model_pipeline.named_steps['model'].predict(X_transformed)[0]
    return round(pred)

# Predict row-wise with MAE-based range
predictions = []
for idx, row in tqdm(df_features[enhanced_features].iterrows(), total=len(df_features), desc="Predicting with MAE bounds"):
    pred = predict_visitors(enhanced_model, pd.DataFrame([row]))
    predictions.append((pred, pred - mae_test, pred + mae_test))

df_features[['PredictedUsers', 'LowerBound', 'UpperBound']] = pd.DataFrame(predictions, index=df_features.index)

StatementMeta(, 6b210051-0b76-4989-9dc5-1bf0aaec88bd, 25, Finished, Available, Finished)

Predicting with MAE bounds: 100%|██████████| 359762/359762 [10:15<00:00, 584.48it/s]


In [24]:
df_features.head()

StatementMeta(, 6b210051-0b76-4989-9dc5-1bf0aaec88bd, 26, Finished, Available, Finished)

,FactLiveKey,ParkKey,DateKey,HourKey,PopularTimesLivePercent,ParkSize,Score,ClassificationScore,BusStopHa,PopHa,Temp,IsRaining,IsSnowing,PredictedUsers,LowerBound,UpperBound
0,1325350010,1776155527,20241219,6,7.000000,532420.043200,29,7,0.143925,36.977463,8.5,1.0,0.0,43,17.013866,68.986134
1,3263572758,2627472210,20250220,7,30.673645,4468.814178,2,1,0.262428,52.201026,7.0,0.0,0.0,21,-4.986134,46.986134
2,116059930,1057569885,20240608,15,63.000000,75056.061370,17,5,0.071722,32.938815,24.0,0.0,0.0,140,114.013866,165.986134
3,3516159715,840186894,20240714,7,7.000000,78590.921970,9,5,0.123167,42.662993,17.1,0.0,0.0,34,8.013866,59.986134
4,3496973816,855495364,20250115,15,38.333736,223436.103400,3,7,0.119527,19.728817,5.7,0.0,0.0,32,6.013866,57.986134


In [25]:
# Final user_predictions table
user_predictions = df_features[[
    'FactLiveKey', 'ParkKey', 'DateKey', 'HourKey'
] + enhanced_features + ['PredictedUsers', 'LowerBound', 'UpperBound']]

# Drop duplicates by FactLiveKey (keep first)
user_predictions = user_predictions.drop_duplicates(subset='FactLiveKey')

# Write to Lakehouse
spark_df_output = spark.createDataFrame(user_predictions)
spark_df_output.write.mode("overwrite").saveAsTable("user_estimates")

StatementMeta(, 6b210051-0b76-4989-9dc5-1bf0aaec88bd, 27, Finished, Available, Finished)

## Get Confidence Intervals

In [ ]:
# import numpy as np
# import pandas as pd

# def compute_ci_per_park_per_day(df, y_pred_full, residuals, n_boot=1000, ci=95):
#     alpha = 100 - ci
#     boot_aggregates = []

#     for i in range(n_boot):
#         noise = np.random.choice(residuals, size=len(y_pred_full), replace=True)
#         simulated = y_pred_full + noise
#         df_sim = pd.DataFrame({
#             'DateKey': df['DateKey'],
#             'ParkKey': df['ParkKey'],
#             'sim': simulated
#         })
#         agg_sim = df_sim.groupby(['DateKey', 'ParkKey'])['sim'].sum().reset_index()
#         boot_aggregates.append(agg_sim['sim'].values)

#     boot_array = np.stack(boot_aggregates, axis=1)
#     lower = np.percentile(boot_array, alpha / 2, axis=1)
#     upper = np.percentile(boot_array, 100 - alpha / 2, axis=1)

#     df_pred = pd.DataFrame({
#         'DateKey': df['DateKey'],
#         'ParkKey': df['ParkKey'],
#         'y_pred': y_pred_full
#     })
#     agg_pred = df_pred.groupby(['DateKey', 'ParkKey'])['y_pred'].sum().reset_index()
#     agg_pred['ci_lower'] = lower
#     agg_pred['ci_upper'] = upper
#     agg_pred['ci_pm'] = (upper - lower) / 2

#     return agg_pred

In [ ]:
# # If you saved X_train and y_train in your pipeline, load and transform them
# # Or just load a subset of training data to compute residuals again

# # Assuming you still have them:
# X_train_trans = enhanced_model.named_steps['prep'].transform(X_enhanced_train)
# y_pred_train = enhanced_model.named_steps['model'].predict(X_train_trans)
# residuals = y_enhanced_train - y_pred_train

In [ ]:
# df_ci = compute_ci_per_park_per_day(
#     df=df_features[['DateKey', 'ParkKey']],
#     y_pred_full=df_features['PredictedUsers'].values,
#     residuals=residuals,
#     n_boot=1000,   # or 500 if you're tight on compute
#     ci=95
# )
